In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import brier_score_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from scipy.stats import uniform,randint
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


In [2]:
data = pd.read_csv('../data/preprocess_data.csv',sep = ',')

In [3]:
pd.options.display.max_columns = None
data.head(5)

,Tournament,Home Team,Away Team,Match Date,Winner Team,Winner Home Or Away,Phase,Final Score,Extra Periods,Team,Opponent Team,Team Points,Team Two Pointers Made,Team Two Pointers Attempted,Team Two Pointers Percenage,Team Three Pointers Made,Team Three Pointers Attempted,Team Three Pointers Percenage,Team Free Throws Made,Team Free Throws Attempted,Team Free Throws Percenage,Team Offensive Rebounds,Team Defensive Rebounds,Team Total Rebounds,Team Assists,Team Steals,Team Turnovers,Team Blocks,Team Blocks Against,Team Personal Fouls,Team Fouls Received,Team Performance Index,Opponent Points,Opponent Two Pointers Made,Opponent Two Pointers Attempted,Opponent Two Pointers Percenage,Opponent Three Pointers Made,Opponent Three Pointers Attempted,Opponent Three Pointers Percenage,Opponent Free Throws Made,Opponent Free Throws Attempted,Opponent Free Throws Percenage,Opponent Offensive Rebounds,Opponent Defensive Rebounds,Opponent Total Rebounds,Opponent Assists,Opponent Steals,Opponent Turnovers,Opponent Blocks,Opponent Blocks Against,Opponent Personal Fouls,Opponent Fouls Received,Opponent Performance Index,Team Result,teamFGA,teamFGM,teamTREB%,teamASST%,teamTS%,teamEFG%,teamOREB%,teamDREB%,teamTO%,teamPoss,teamSTL%,teamBLK%,teamBLKR,teamPPS,teamPlay%,teamAR,teamAST/TO,teamSTL/TO,teamFIC,teamOrtg,opptFGA,opptPoss,teamDrtg,teamEDiff,team_Game_Score,opptFGM,opptTREB%,opptASST%,opptTS%,opptEFG%,opptOREB%,opptDREB%,opptTO%,opptSTL%,opptBLK%,opptBLKR,opptPPS,opptFIC,opptOrtg,opptDrtg,opptEDiff,opptPlay%,opptAR,opptAST/TO,opptSTL/TO,oppt_Game_Score,Points difference,Finals,Last 32,Playoffs,Quarter-Finals,Regular Season,Semifinals,Top 16
0,Basket League,KAOD,Aris,2013-10-12,Aris,Away,Regular Season,58-66,0.0,KAOD,Aris,58.0,20.0,43.0,0.46512,1.0,10.0,0.10000,15.0,26.0,0.57692,12.0,22.0,34.0,6.0,6.0,17.0,3.0,3.0,22.0,22.0,40.0,66.0,15.0,33.0,0.45455,9.0,23.0,0.39130,9.0,19.0,0.47368,10.0,18.0,28.0,11.0,8.0,14.0,3.0,3.0,22.0,22.0,57.0,0.0,53.0,21.0,54.838710,28.571429,45.003104,40.566038,40.000000,68.750000,20.874263,69.44,8.640553,4.320276,9.090909,1.094340,36.206897,9.293680,0.352941,0.352941,24.000,83.525346,56.0,68.36,96.547689,-13.022343,26.4,24.0,45.161290,45.833333,51.274083,50.892857,31.250000,60.000000,17.866258,11.702750,4.388531,6.976744,1.178571,37.375,96.547689,83.525346,13.022343,40.000000,15.143172,0.785714,0.571429,39.8,-8.0,0,0,0,0,1,0,0
1,Basket League,PAOK,Kolossos Rhodes,2013-10-12,PAOK,Home,Regular Season,88-81,0.0,PAOK,Kolossos Rhodes,88.0,17.0,35.0,0.48571,10.0,26.0,0.38462,24.0,26.0,0.92308,9.0,22.0,31.0,22.0,7.0,18.0,3.0,1.0,23.0,23.0,96.0,81.0,26.0,42.0,0.61905,5.0,16.0,0.31250,14.0,21.0,0.66667,6.0,25.0,31.0,5.0,8.0,18.0,1.0,3.0,24.0,22.0,69.0,1.0,61.0,27.0,50.000000,81.481481,60.739923,52.459016,26.470588,78.571429,19.902698,81.44,8.595285,3.683694,7.142857,1.442623,38.571429,24.564538,1.222222,0.388889,60.500,108.055010,58.0,79.24,102.221100,5.833909,65.5,31.0,50.000000,16.129032,60.232005,57.758621,21.428571,73.529412,21.116847,10.095911,1.261989,2.857143,1.396552,38.375,102.221100,108.055010,-5.833909,44.285714,6.967670,0.277778,0.444444,46.3,7.0,0,0,0,0,1,0,0
2,Liga ACB,Bilbao,Zaragoza,2013-10-12,Zaragoza,Away,Regular Season,77-86,0.0,Bilbao,Zaragoza,77.0,24.0,46.0,0.52000,5.0,12.0,0.42000,14.0,20.0,0.70000,13.0,19.0,32.0,15.0,7.0,12.0,1.0,1.0,18.0,23.0,89.0,86.0,25.0,39.0,0.64000,10.0,23.0,0.43000,6.0,10.0,0.60000,9.0,20.0,29.0,16.0,10.0,10.0,1.0,1.0,23.0,18.0,95.0,0.0,58.0,29.0,52.459016,51.724138,57.634731,54.310345,39.393939,67.857143,15.228426,65.80,10.638298,1.519757,2.564103,1.327586,50.877193,19.685039,1.250000,0.583333,55.250,117.021277,62.0,67.40,127.596439,-10.575163,59.4,35.0,47.540984,45.714286,64.759036,64.516129,32.142857,60.606061,13.089005,14.836795,1.483680,2.173913,1.387097,65.250,127.596439,117.021277,10.575163,55.555556,19.138756,1.600000,1.000000,70.0,-9.0,0,0,0,0,1,0,0
3,Basket League,Trikala,Kifisia,2013-10-12,Kifisia,Away,Regular Season,64-69,0.0,Trikala,Kifisia,64.0,21.0,41.0,0.51220,

In [4]:
data["Tournament"].unique()

array(['Basket League', 'Liga ACB', 'Eurocup', 'Euroleague'], dtype=object)

# Teams as Dummy Variables

In [5]:
baseline_data = data[["Tournament","Match Date","Home Team","Away Team","Team Result","Winner Team","Phase"]]
baseline_data.head()

,Tournament,Match Date,Home Team,Away Team,Team Result,Winner Team,Phase
0,Basket League,2013-10-12,KAOD,Aris,0.0,Aris,Regular Season
1,Basket League,2013-10-12,PAOK,Kolossos Rhodes,1.0,PAOK,Regular Season
2,Liga ACB,2013-10-12,Bilbao,Zaragoza,0.0,Zaragoza,Regular Season
3,Basket League,2013-10-12,Trikala,Kifisia,0.0,Kifisia,Regular Season
4,Liga ACB,2013-10-12,Tenerife,Obradoiro,1.0,Tenerife,Regular Season


## Greek Basket League 

In [6]:
greek_baseline_data = baseline_data[["Tournament","Match Date","Home Team","Away Team","Team Result","Winner Team","Phase"]][baseline_data["Tournament"]=='Basket League'].reset_index(drop = True)
greek_baseline_data.head()

,Tournament,Match Date,Home Team,Away Team,Team Result,Winner Team,Phase
0,Basket League,2013-10-12,KAOD,Aris,0.0,Aris,Regular Season
1,Basket League,2013-10-12,PAOK,Kolossos Rhodes,1.0,PAOK,Regular Season
2,Basket League,2013-10-12,Trikala,Kifisia,0.0,Kifisia,Regular Season
3,Basket League,2013-10-13,Panelefsiniakos,Panionios,0.0,Panionios,Regular Season
4,Basket League,2013-10-13,Rethymno,Ikaros,1.0,Rethymno,Regular Season


In [7]:
greece_home = pd.get_dummies(greek_baseline_data["Home Team"]).reset_index(drop = True)
greece_away = -1*pd.get_dummies(greek_baseline_data["Away Team"]).reset_index(drop = True)
greece_x = pd.concat([greece_home,greece_away],axis=1).groupby(level=0, axis=1).sum()
greece_y = greek_baseline_data["Team Result"]

In [8]:
greek_baseline_data['Phase'].unique()

array(['Regular Season', 'Playoffs'], dtype=object)

In [9]:
x_structure_greece = greece_x.iloc[:len(greek_baseline_data[greek_baseline_data["Match Date"]<"2017-08-01"])]
y_structure_greece = greece_y.iloc[:len(greek_baseline_data[greek_baseline_data["Match Date"]<"2017-08-01"])]

start = int(len(greek_baseline_data[greek_baseline_data["Match Date"]<"2017-08-01"]))
end = start + int(len(greek_baseline_data[(greek_baseline_data["Match Date"]>"2017-08-01") & (greek_baseline_data['Phase'].isin(['Regular Season']))]))

x_train_greece = greece_x.iloc[:end]
y_train_greece = greece_y.iloc[:end]


x_test_greece = greece_x.iloc[end:]
y_test_greece = greece_y.iloc[end:]

###  Logistic Regression

In [10]:
lgr = LogisticRegression(C = 0.3839820885947254,penalty = 'l1', solver= 'liblinear').fit(x_train_greece, y_train_greece)
y_pre_proba_lgr=lgr.predict_proba(x_test_greece)
print("Logistic Regression")
print()
print("Test BS:",round(brier_score_loss(y_test_greece,y_pre_proba_lgr[:,1]),3))
print()
y_pre_lgr=np.where(pd.Series(y_pre_proba_lgr[:,1])>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_lgr,y_test_greece),3))
print()
print("Test F1-score:",round(f1_score(y_pre_lgr,y_test_greece),3))


Logistic Regression

Test BS: 0.143

Test Accuracy: 0.727

Test F1-score 0.8


In [11]:
coef_dict = {}
for coef, feat in zip(lgr.coef_[:].tolist()[0],greece_x.columns.values):
    coef_dict[feat] = coef
coef1=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef1=coef1.iloc[(-coef1[0].abs()).argsort()].reset_index()

In [12]:
sorted_coef1

,index,0
0,Olympiacos,3.535727
1,Panathinaikos,3.411626
2,AEK,1.276698
3,PAOK,0.951240
4,Aris,0.800854
5,Doxa Lefkadas,-0.774154
6,Panelefsiniakos,-0.591308
7,Kavala,-0.568842
8,Promitheas Patras,0.413823
9,Trikala,-0.379628


In [13]:
lgr.intercept_

array([0.82164137])

## Random Forest Classifier

In [14]:
rfc = RandomForestClassifier(max_depth= 38, max_features='sqrt', min_samples_leaf=0.0022712801155358453,
                             min_samples_split=0.009889030194338, n_estimators=53
                             ,random_state= 1).fit(x_train_greece, y_train_greece)

y_pre_proba_rfc=rfc.predict_proba(x_test_greece)
print("Random Forest Classifier")
print()
print("Test BS:",round(brier_score_loss(y_test_greece,y_pre_proba_rfc[:,1]),3))
print()
y_pre_rfc=np.where(pd.Series(y_pre_proba_rfc[:,1])>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_rfc,y_test_greece),3))
print()
print("Test F1-score:",round(f1_score(y_pre_rfc,y_test_greece),3))

Random Forest Classifier

Test BS: 0.151

Test Accuracy: 0.682

Test F1-score 0.741


In [15]:
coef_dict = {}
for coef, feat in zip(rfc.feature_importances_[:].tolist(),greece_x.columns.values):
    coef_dict[feat] = coef
coef2=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef2=coef2.iloc[(-coef2[0].abs()).argsort()].reset_index()
sorted_coef2

,index,0
0,Panathinaikos,0.289872
1,Olympiacos,0.285092
2,AEK,0.048372
3,Trikala,0.041963
4,PAOK,0.031702
5,Koroivos,0.030695
6,Aris,0.030650
7,Panelefsiniakos,0.027916
8,Doxa Lefkadas,0.022918
9,Panionios,0.022221


## xgboost

In [16]:
alg = XGBClassifier(colsample_bytree = 0.7120387144249468, gamma = 1.173482282705095, learning_rate = 0.18114216198386113,
                    max_depth= 41, n_estimators= 88, reg_lambda= 0.33962213881868086, subsample= 0.16627201904949962, 
                    objective='binary:logistic').fit(x_train_greece, y_train_greece)




y_pre_proba_alg=alg.predict_proba(x_test_greece)
print("xgboost")
print()
print("Test BS:",round(brier_score_loss(y_test_greece,y_pre_proba_alg[:,1]),3))
print()
y_pre_alg=np.where(pd.Series(y_pre_proba_alg[:,1])>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_alg,y_test_greece),3))
print()
print("Test F1-score:",round(f1_score(y_pre_alg,y_test_greece),3))


xgboost

Test BS: 0.14

Test Accuracy: 0.727

Test F1-score 0.8


In [17]:
coef_dict = {}
for coef, feat in zip(alg.feature_importances_[:].tolist(),greece_x.columns.values):
    coef_dict[feat] = coef
coef3=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef3=coef3.iloc[(-coef3[0].abs()).argsort()].reset_index()
sorted_coef3

,index,0
0,Kavala,0.080205
1,Olympiacos,0.071937
2,Panathinaikos,0.068753
3,Panelefsiniakos,0.067958
4,Panionios,0.056043
5,AEK,0.052077
6,Koroivos,0.050583
7,Arkadikos,0.050234
8,Aris,0.050126
9,Trikala,0.046963


# Ensemble

In [18]:
ens_proba=(y_pre_proba_lgr[:,1]+y_pre_proba_rfc[:,1]+y_pre_proba_alg[:,1])/3


print("------Ensemble 3 classifier-------")
print()
print("Test BS:",round(brier_score_loss(y_test_greece,ens_proba),3))
print()
y_pre_ens=np.where(pd.Series(ens_proba)>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_ens,y_test_greece),3))
print()
print("Test F1-score:",round(f1_score(y_pre_ens,y_test_greece),3))


------Ensemble 3 classifier-------

Test BS: 0.142

Test Accuracy: 0.727

Test F1-score 0.8


In [19]:
matches = greek_baseline_data[(greek_baseline_data["Match Date"]>"2017-08-01") & (greek_baseline_data['Phase'].isin(['Playoffs']))]
matches['Probability']=ens_proba
matches['Prediction']=np.where(matches['Probability']>0.5,1,0)
matches['BS']= (matches['Team Result'] -  matches['Probability'])**2
matches

,Tournament,Match Date,Home Team,Away Team,Team Result,Winner Team,Phase,Probability,Prediction,BS
1007,Basket League,2018-05-16,PAOK,AEK,1.0,PAOK,Playoffs,0.562269,1,0.191609
1008,Basket League,2018-05-16,Panathinaikos,Kolossos Rhodes,1.0,Panathinaikos,Playoffs,0.977662,1,0.000499
1009,Basket League,2018-05-16,Promitheas Patras,Lavrio,1.0,Promitheas Patras,Playoffs,0.725633,1,0.075277
1010,Basket League,2018-05-16,Olympiacos,Kymi,1.0,Olympiacos,Playoffs,0.979513,1,0.000420
1011,Basket League,2018-05-19,Kolossos Rhodes,Panathinaikos,0.0,Panathinaikos,Playoffs,0.102067,0,0.010418
1012,Basket League,2018-05-19,Lavrio,Promitheas Patras,0.0,Promitheas Patras,Playoffs,0.566444,1,0.320859
1013,Basket League,2018-05-19,Kymi,Olympiacos,0.0,Olympiacos,Playoffs,0.087648,0,0.007682
1014,Basket League,2018-05-19,AEK,PAOK,0.0,PAOK,Playoffs,0.735298,1,0.540664
1015,Basket League,2018-05-24,Olympiacos,Promitheas Patras,1.0,Olympiacos,Playoffs,0.942073,1,0.003355
1016,Basket League,2018-05-24,Panathinaikos,PAOK,1.0,Panathinaikos,Playoffs,0.957906,1,0.001772


# Liga ACB

In [20]:
spain_baseline_data = baseline_data[["Tournament","Match Date","Home Team","Away Team","Team Result","Winner Team","Phase"]][baseline_data["Tournament"]=='Liga ACB'].reset_index(drop = True)
spain_baseline_data.head()

,Tournament,Match Date,Home Team,Away Team,Team Result,Winner Team,Phase
0,Liga ACB,2013-10-12,Bilbao,Zaragoza,0.0,Zaragoza,Regular Season
1,Liga ACB,2013-10-12,Tenerife,Obradoiro,1.0,Tenerife,Regular Season
2,Liga ACB,2013-10-13,Manresa,Joventut,1.0,Manresa,Regular Season
3,Liga ACB,2013-10-13,Baskonia,Gran Canaria,0.0,Gran Canaria,Regular Season
4,Liga ACB,2013-10-13,Gipuzkoa,Real Betis,1.0,Gipuzkoa,Regular Season


In [21]:
spain_home = pd.get_dummies(spain_baseline_data["Home Team"]).reset_index(drop = True)
spain_away = -1*pd.get_dummies(spain_baseline_data["Away Team"]).reset_index(drop = True)
spain_x = pd.concat([spain_home,spain_away],axis=1).groupby(level=0, axis=1).sum()
spain_y = spain_baseline_data["Team Result"]

In [22]:
spain_x.head()

,Andorra,Barcelona,Baskonia,Bilbao,Burgos,Estudiantes,Fuenlabrada,Gipuzkoa,Gran Canaria,Joventut,Manresa,Murcia,Obradoiro,Real Betis,Real Madrid,Tenerife,Unicaja Malaga,Valencia,Valladolid,Zaragoza
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1
1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0


In [23]:
spain_baseline_data['Phase'].unique()

array(['Regular Season', 'Playoffs'], dtype=object)

In [24]:
x_structure_spain = spain_x.iloc[:len(spain_baseline_data[spain_baseline_data["Match Date"]<"2017-08-01"])]
y_structure_spain = spain_y.iloc[:len(spain_baseline_data[spain_baseline_data["Match Date"]<"2017-08-01"])]

start = int(len(spain_baseline_data[spain_baseline_data["Match Date"]<"2017-08-01"]))
end = start + int(len(spain_baseline_data[(spain_baseline_data["Match Date"]>"2017-08-01") & (spain_baseline_data['Phase'].isin(['Regular Season']))]))

x_train_spain = spain_x.iloc[:end]
y_train_spain = spain_y.iloc[:end]


x_test_spain = spain_x.iloc[end:]
y_test_spain = spain_y.iloc[end:]

###  Logistic Regression

In [25]:
lgr = LogisticRegression(solver='liblinear',C = 0.6765129921671309, penalty = 'l1').fit(x_train_spain, y_train_spain)
y_pre_proba_lgr=lgr.predict_proba(x_test_spain)
print("Logistic Regression")
print()
print("Test BS:",round(brier_score_loss(y_test_spain,y_pre_proba_lgr[:,1]),3))
print()
y_pre_lgr=np.where(pd.Series(y_pre_proba_lgr[:,1])>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_lgr,y_test_spain),3))
print()
print("Test F1-score:",round(f1_score(y_pre_lgr,y_test_spain),3))


Logistic Regression

Test BS: 0.227

Test Accuracy: 0.619

Test F1-score 0.667


In [26]:
coef_dict = {}
for coef, feat in zip(lgr.coef_[:].tolist()[0],spain_x.columns.values):
    coef_dict[feat] = coef
coef2=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef2=coef2.iloc[(-coef2[0].abs()).argsort()].reset_index()

In [27]:
sorted_coef2

,index,0
0,Real Madrid,2.192283
1,Valladolid,-1.820969
2,Barcelona,1.654085
3,Valencia,1.507102
4,Unicaja Malaga,1.013158
5,Baskonia,1.011561
6,Manresa,-0.842087
7,Gran Canaria,0.797715
8,Tenerife,0.409584
9,Gipuzkoa,-0.357511


In [28]:
lgr.intercept_

array([0.63741694])

## Random Forest Classifier

In [29]:
rfc = RandomForestClassifier(max_depth= 39, max_features='sqrt', min_samples_leaf=0.0009330471732301171,
                             min_samples_split=0.075312968832076, n_estimators=73,
                             random_state=1).fit(x_train_spain, y_train_spain)

y_pre_proba_rfc=rfc.predict_proba(x_test_spain)
print("Random Forest Classifier")
print()
print("Test BS:",round(brier_score_loss(y_test_spain,y_pre_proba_rfc[:,1]),3))
print()
y_pre_rfc=np.where(pd.Series(y_pre_proba_rfc[:,1])>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_rfc,y_test_spain),3))
print()
print("Test F1-score:",round(f1_score(y_pre_rfc,y_test_spain),3))

Random Forest Classifier

Test BS: 0.24

Test Accuracy: 0.714

Test F1-score 0.769


In [30]:
coef_dict = {}
for coef, feat in zip(rfc.feature_importances_[:].tolist(),spain_x.columns.values):
    coef_dict[feat] = coef
coef2=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef2=coef2.iloc[(-coef2[0].abs()).argsort()].reset_index()
sorted_coef2

,index,0
0,Real Madrid,0.237745
1,Barcelona,0.119867
2,Valencia,0.117710
3,Manresa,0.090713
4,Unicaja Malaga,0.053261
5,Baskonia,0.051651
6,Valladolid,0.051592
7,Gipuzkoa,0.050253
8,Real Betis,0.042584
9,Gran Canaria,0.033088


## xgboost

In [31]:
alg = XGBClassifier(colsample_bytree= 0.6188949028768943, gamma= 2.819830261508139, learning_rate= 0.36256801131990146,
                    max_depth= 21, n_estimators= 86, reg_lambda= 0.09211406347716067, subsample= 0.5079441274104476,
                    objective='binary:logistic').fit(x_train_spain, y_train_spain)

y_pre_proba_alg=alg.predict_proba(x_test_spain)
print("xgboost")
print()
print("Test BS:",round(brier_score_loss(y_test_spain,y_pre_proba_alg[:,1]),3))
print()
y_pre_alg=np.where(pd.Series(y_pre_proba_alg[:,1])>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_alg,y_test_spain),3))
print()
print("Test F1-score:",round(f1_score(y_pre_alg,y_test_spain),3))


xgboost

Test BS: 0.248

Test Accuracy: 0.714

Test F1-score 0.769


In [32]:
coef_dict = {}
for coef, feat in zip(alg.feature_importances_[:].tolist(),spain_x.columns.values):
    coef_dict[feat] = coef
coef3=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef3=coef3.iloc[(-coef3[0].abs()).argsort()].reset_index()
sorted_coef3

,index,0
0,Real Madrid,0.081206
1,Valencia,0.076525
2,Estudiantes,0.063967
3,Manresa,0.062146
4,Barcelona,0.060338
5,Gipuzkoa,0.057723
6,Obradoiro,0.050125
7,Bilbao,0.049827
8,Zaragoza,0.049434
9,Tenerife,0.049139


In [33]:
ens_proba=(y_pre_proba_lgr[:,1]+y_pre_proba_rfc[:,1]+y_pre_proba_alg[:,1])/3


print("------Ensemble 3 classifier-------")
print()
print("Test BS:",round(brier_score_loss(y_test_spain,ens_proba),3))
print()
y_pre_ens=np.where(pd.Series(ens_proba)>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_ens,y_test_spain),3))
print()
print("Test F1-score:",round(f1_score(y_pre_ens,y_test_spain),3))


------Ensemble 3 classifier-------

Test BS: 0.237

Test Accuracy: 0.714

Test F1-score 0.769


In [34]:
matches = spain_baseline_data[(spain_baseline_data["Match Date"]>"2017-08-01") & (spain_baseline_data['Phase'].isin(['Playoffs']))]
matches['Probability']=y_pre_proba_lgr[:,1]
matches['Prediction']=np.where(matches['Probability']>0.5,1,0)
matches['BS']= (matches['Team Result'] -  matches['Probability'])**2
matches

,Tournament,Match Date,Home Team,Away Team,Team Result,Winner Team,Phase,Probability,Prediction,BS
1586,Liga ACB,2018-05-27,Barcelona,Andorra,0.0,Andorra,Playoffs,0.900674,1,0.811214
1587,Liga ACB,2018-05-27,Baskonia,Unicaja Malaga,1.0,Baskonia,Playoffs,0.653808,1,0.119849
1588,Liga ACB,2018-05-27,Real Madrid,Tenerife,1.0,Real Madrid,Playoffs,0.918348,1,0.006667
1589,Liga ACB,2018-05-28,Valencia,Gran Canaria,1.0,Valencia,Playoffs,0.793607,1,0.042598
1590,Liga ACB,2018-05-29,Tenerife,Real Madrid,0.0,Real Madrid,Playoffs,0.241352,0,0.058251
1591,Liga ACB,2018-05-29,Unicaja Malaga,Baskonia,0.0,Baskonia,Playoffs,0.654530,1,0.428410
1592,Liga ACB,2018-05-30,Andorra,Barcelona,0.0,Barcelona,Playoffs,0.282945,0,0.080058
1593,Liga ACB,2018-05-30,Gran Canaria,Valencia,1.0,Gran Canaria,Playoffs,0.482015,0,0.268308
1594,Liga ACB,2018-06-01,Valencia,Gran Canaria,0.0,Gran Canaria,Playoffs,0.793607,1,0.629812
1595,Liga ACB,2018-06-01,Barcelona,Andorra,1.0,Barcelona,Playoffs,0.900674,1,0.009866


In [35]:
np.mean(matches['BS'])

0.22740994662159691

# Euroleague

In [36]:
el_baseline_data = baseline_data[["Tournament","Match Date","Home Team","Away Team","Team Result","Winner Team","Phase"]][baseline_data["Tournament"]=='Euroleague'].reset_index(drop = True)
el_baseline_data.head()

,Tournament,Match Date,Home Team,Away Team,Team Result,Winner Team,Phase
0,Euroleague,2013-10-16,Bamberg,Strasbourg,1.0,Bamberg,Regular Season
1,Euroleague,2013-10-16,Siena,Galatasaray,0.0,Galatasaray,Regular Season
2,Euroleague,2013-10-17,Barcelona,Partizan,1.0,Barcelona,Regular Season
3,Euroleague,2013-10-17,Nanterre,CSKA Moscow,0.0,CSKA Moscow,Regular Season
4,Euroleague,2013-10-17,Anadolu Efes,Armani Milano,1.0,Anadolu Efes,Regular Season


In [37]:
el_home = pd.get_dummies(el_baseline_data["Home Team"]).reset_index(drop = True)
el_away = -1*pd.get_dummies(el_baseline_data["Away Team"]).reset_index(drop = True)
el_x = pd.concat([el_home,el_away],axis=1).groupby(level=0, axis=1).sum()
el_y = el_baseline_data["Team Result"]

In [38]:
el_x.head()

,Alba Berlin,Anadolu Efes,Armani Milano,Bamberg,Barcelona,Baskonia,Bayern Munich,Budivelnyk,CSKA Moscow,Cedevita,Crvena Zvezda,Darussafaka,Dinamo Sassari,Fenerbahce,Galatasaray,Khimki,Limoges,Lokomotiv Kuban,Maccabi Tel Aviv,Nanterre,Neptunas,Nizhny Novgorod,Olympiacos,Panathinaikos,Partizan,Pinar Karsiyaka,Real Madrid,Rytas Vilnius,Siena,Strasbourg,Turow Zgorzelec,Unicaja Malaga,Unics Kazan,Valencia,Zalgiris,Zielona Gora
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [39]:
el_baseline_data['Phase'].unique()

array(['Regular Season', 'Top 16', 'Playoffs', 'Final Four'], dtype=object)

In [40]:
x_structure_el = el_x.iloc[:len(el_baseline_data[el_baseline_data["Match Date"]<"2017-08-01"])]
y_structure_el = el_y.iloc[:len(el_baseline_data[el_baseline_data["Match Date"]<"2017-08-01"])]

start = int(len(el_baseline_data[el_baseline_data["Match Date"]<"2017-08-01"]))
end = start + int(len(el_baseline_data[(el_baseline_data["Match Date"]>"2017-08-01") & (el_baseline_data['Phase'].isin(['Regular Season', 'Top 16']))]))

x_train_el = el_x.iloc[:end]
y_train_el = el_y.iloc[:end]


x_test_el = el_x.iloc[end:]
y_test_el = el_y.iloc[end:]

###  Logistic Regression

In [41]:
lgr = LogisticRegression(solver='liblinear' , C= 0.7847137962592965, penalty='l1').fit(x_train_el, y_train_el)
y_pre_proba_lgr=lgr.predict_proba(x_test_el)
print("Logistic Regression")
print()
print("Test BS:",round(brier_score_loss(y_test_el,y_pre_proba_lgr[:,1]),3))
print()
y_pre_lgr=np.where(pd.Series(y_pre_proba_lgr[:,1])>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_lgr,y_test_el),3))
print()
print("Test F1-score:",round(f1_score(y_pre_lgr,y_test_el),3))


Logistic Regression

Test BS: 0.242

Test Accuracy: 0.6

Test F1-score 0.714


In [42]:
coef_dict = {}
for coef, feat in zip(lgr.coef_[:].tolist()[0],el_x.columns.values):
    coef_dict[feat] = coef
coef3=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef3=coef3.iloc[(-coef3[0].abs()).argsort()].reset_index()

In [43]:
sorted_coef3.head(50)

,index,0
0,Dinamo Sassari,-1.979196
1,CSKA Moscow,1.872636
2,Fenerbahce,1.411630
3,Real Madrid,1.321749
4,Olympiacos,1.195239
5,Lokomotiv Kuban,0.861180
6,Turow Zgorzelec,-0.856414
7,Zielona Gora,-0.856121
8,Barcelona,0.820067
9,Rytas Vilnius,-0.766547


In [44]:
lgr.intercept_

array([0.62443411])

## Random Forest Classifier

In [45]:
rfc = RandomForestClassifier(max_depth= 36, max_features='log2', min_samples_leaf=0.002390948900012013,
                             min_samples_split=0.0096605683266830, n_estimators=73,
                             random_state=1).fit(x_train_el, y_train_el)

y_pre_proba_rfc=rfc.predict_proba(x_test_el)
print("Random Forest Classifier")
print()
print("Test BS:",round(brier_score_loss(y_test_el,y_pre_proba_rfc[:,1]),3))
print()
y_pre_rfc=np.where(y_pre_proba_rfc[:,1] > 0.5, 1, 0)
print("Test Accuracy:",round(accuracy_score(y_pre_rfc ,y_test_el),3))
print()
print("Test F1-score:",round(f1_score(y_pre_rfc,y_test_el),3))

Random Forest Classifier

Test BS: 0.239

Test Accuracy: 0.55

Test F1-score 0.667


In [46]:
coef_dict = {}
for coef, feat in zip(rfc.feature_importances_[:].tolist(),el_x.columns.values):
    coef_dict[feat] = coef
coef2=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef2=coef2.iloc[(-coef2[0].abs()).argsort()].reset_index()
sorted_coef2

,index,0
0,CSKA Moscow,0.185142
1,Fenerbahce,0.107072
2,Real Madrid,0.081984
3,Olympiacos,0.060826
4,Dinamo Sassari,0.050181
5,Panathinaikos,0.038085
6,Cedevita,0.036964
7,Barcelona,0.033836
8,Zielona Gora,0.030140
9,Armani Milano,0.025312


## xgboost

In [47]:
alg = XGBClassifier(colsample_bytree= 0.897167919370632, gamma= 0.835671500072126, learning_rate= 0.3887154858505634,
                    max_depth= 1, n_estimators= 99, reg_lambda= 0.7909237442174416, subsample= 0.963659261900305,
                    objective='binary:logistic').fit(x_train_el, y_train_el)

y_pre_proba_alg=alg.predict_proba(x_test_el)
print("xgboost")
print()
print("Test BS:",round(brier_score_loss(y_test_el,y_pre_proba_alg[:,1]),3))
print()
y_pre_alg=np.where(y_pre_proba_alg[:,1] > 0.5, 1, 0)
print("Test Accuracy:",round(accuracy_score(y_pre_alg ,y_test_el),3))
print()
print("Test F1-score:",round(f1_score(y_pre_alg,y_test_el),3))


xgboost

Test BS: 0.242

Test Accuracy: 0.6

Test F1-score 0.714


In [48]:
coef_dict = {}
for coef, feat in zip(alg.feature_importances_[:].tolist(),el_x.columns.values):
    coef_dict[feat] = coef
coef3=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef3=coef3.iloc[(-coef3[0].abs()).argsort()].reset_index()
sorted_coef3

,index,0
0,CSKA Moscow,0.107204
1,Fenerbahce,0.075837
2,Real Madrid,0.070833
3,Olympiacos,0.062947
4,Zielona Gora,0.062656
5,Panathinaikos,0.053514
6,Barcelona,0.039293
7,Cedevita,0.038382
8,Dinamo Sassari,0.035913
9,Lokomotiv Kuban,0.032985


In [49]:
ens_proba=(y_pre_proba_lgr[:,1]+y_pre_proba_rfc[:,1]+y_pre_proba_alg[:,1])/3


print("------Ensemble 3 classifier-------")
print()
print("Test RPS:",round(brier_score_loss(y_test_el,ens_proba),3))
print()
y_pre_ens=np.where(pd.Series(ens_proba)>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_ens,y_test_el),3))
print()
print("Test F1-score:",round(f1_score(y_pre_ens,y_test_el),3))


------Ensemble 3 classifier-------

Test RPS: 0.24

Test Accuracy: 0.6

Test F1-score 0.714


# Eurocup

In [50]:
ec_baseline_data = baseline_data[["Tournament","Match Date","Home Team","Away Team","Team Result","Winner Team","Phase"]][baseline_data["Tournament"]=='Eurocup'].reset_index(drop = True)
ec_baseline_data.head()

,Tournament,Match Date,Home Team,Away Team,Team Result,Winner Team,Phase
0,Eurocup,2013-10-15,Bonn,Alba Berlin,0.0,Alba Berlin,Regular Season
1,Eurocup,2013-10-15,Banvit,Kalev Cramo,1.0,Banvit,Regular Season
2,Eurocup,2013-10-15,PAOK,Buducnost,1.0,PAOK,Regular Season
3,Eurocup,2013-10-15,Bilbao,Cedevita,1.0,Bilbao,Regular Season
4,Eurocup,2013-10-15,Paris Levallois,Valencia,0.0,Valencia,Regular Season


In [51]:
ec_home = pd.get_dummies(ec_baseline_data["Home Team"]).reset_index(drop = True)
ec_away = -1*pd.get_dummies(ec_baseline_data["Away Team"]).reset_index(drop = True)
ec_x = pd.concat([ec_home,ec_away],axis=1).groupby(level=0, axis=1).sum()
ec_y = ec_baseline_data["Team Result"]

In [52]:
ec_x.head()

,AEK,ASVEL Villeurbanne,Alba Berlin,Alba Fehervar,Andorra,Aris,Armani Milano,Artland Dragons,Avtodor Saratov,Bamberg,Banvit,Bayern Munich,Besiktas,Bilbao,Bisons Loimaa,Bonn,Brindisi,Budivelnyk,Buducnost,Cantu,Cedevita,Chalon Saone,Cibona,Crvena Zvezda,Csu Asesoft Ploiesti,Darussafaka,Dijon,Dinamo Sassari,Fuenlabrada,Galatasaray,Gran Canaria,Gravelines,Hapoel Jerusalem,Igokea,Kalev Cramo,Khimik,Khimki,Krasny Oktyabr,Le Mans,Lietkabelis,Limoges,Lokomotiv Kuban,Ludwigsburg,Lukoil Academic,MZT Skopje,Maccabi Haifa,Maccabi Tel Aviv,Mons,Murcia,Nancy,Nanterre,Neptunas,Nizhny Novgorod,Nymburk,Oldenburg,Olimpija Ljubljana,Oostende,PAOK,Panionios,Paris Levallois,Partizan,Pinar Karsiyaka,Radnicki,Real Betis,Reggio Emilia,Rytas Vilnius,Siena,Spartak St Petersburg,Spirou Charleroi,Steaua Bucharest,Strasbourg,Szolnoki,Ted Ankara Kolejliler,Tofas Bursa,Trabzonspor,Trento,Turin,Turow Zgorzelec,Ulm,Unicaja Malaga,Unics Kazan,VEF Riga,Valencia,Varese,Venezia,Ventspils,Virtus Roma,Zaragoza,Zenit St Petersburg,Zielona Gora
0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0


In [53]:
ec_baseline_data['Phase'].unique()

array(['Regular Season', 'Last 32', 'Quarter-Finals', 'Semifinals',
       'Finals', 'Top 16'], dtype=object)

In [54]:
x_structure_ec = ec_x.iloc[:len(ec_baseline_data[ec_baseline_data["Match Date"]<"2017-08-01"])]
y_structure_ec = ec_y.iloc[:len(ec_baseline_data[ec_baseline_data["Match Date"]<"2017-08-01"])]

start = int(len(ec_baseline_data[ec_baseline_data["Match Date"]<"2017-08-01"]))
end = start + int(len(ec_baseline_data[(ec_baseline_data["Match Date"]>"2017-08-01") & (ec_baseline_data['Phase'].isin(['Regular Season','Last 32', 'Top 16']))]))

x_train_ec = ec_x.iloc[:end]
y_train_ec = ec_y.iloc[:end]


x_test_ec = ec_x.iloc[end:]
y_test_ec = ec_y.iloc[end:]

###  Logistic Regression

In [55]:
lgr = LogisticRegression(solver='liblinear',C= 0.8479198446641489, penalty= 'l1').fit(x_train_ec, y_train_ec)
y_pre_proba_lgr=lgr.predict_proba(x_test_ec)
print("Logistic Regression")
print()
print("Test BS:",round(brier_score_loss(y_test_ec,y_pre_proba_lgr[:,1]),3))
print()
y_pre_lgr=np.where(pd.Series(y_pre_proba_lgr[:,1])>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_lgr,y_test_ec),3))
print()
print("Test F1-score:",round(f1_score(y_pre_lgr,y_test_ec),3))


Logistic Regression

Test BS: 0.161

Test Accuracy: 0.75

Test F1-score 0.818


In [56]:
coef_dict = {}
for coef, feat in zip(lgr.coef_[:].tolist()[0],ec_x.columns.values):
    coef_dict[feat] = coef
coef4=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef4=coef4.iloc[(-coef4[0].abs()).argsort()].reset_index()

In [57]:
sorted_coef4.head(50)

,index,0
0,MZT Skopje,-2.060959
1,Cibona,-1.862996
2,Lokomotiv Kuban,1.840229
3,Khimki,1.695401
4,Artland Dragons,-1.631507
5,Darussafaka,1.565266
6,Bayern Munich,1.524909
7,Valencia,1.274799
8,Unics Kazan,1.263567
9,Gran Canaria,1.214537


In [58]:
lgr.intercept_

array([0.53294419])

## Random Forest Classifier

In [59]:
rfc = RandomForestClassifier(max_depth= 38, max_features='sqrt', min_samples_leaf=0.0037012163422379674,
                             min_samples_split=0.251132570859313, n_estimators=41,
                             random_state=1).fit(x_train_ec, y_train_ec)

y_pre_proba_rfc=rfc.predict_proba(x_test_ec)
print("Random Forest Classifier")
print()
print("Test BS:",round(brier_score_loss(y_test_ec,y_pre_proba_rfc[:,1]),3))
print()
y_pre_rfc=np.where(y_pre_proba_rfc[:,1] > 0.5, 1, 0)
print("Test Accuracy:",round(accuracy_score(y_pre_rfc ,y_test_ec),3))
print()
print("Test F1-score:",round(f1_score(y_pre_rfc,y_test_ec),3))

Random Forest Classifier

Test BS: 0.246

Test Accuracy: 0.5

Test F1-score 0.556


In [60]:
coef_dict = {}
for coef, feat in zip(rfc.feature_importances_[:].tolist(),ec_x.columns.values):
    coef_dict[feat] = coef
coef2=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef2=coef2.iloc[(-coef2[0].abs()).argsort()].reset_index()
sorted_coef2

,index,0
0,Lokomotiv Kuban,0.110517
1,Unics Kazan,0.075096
2,Khimki,0.072364
3,Gran Canaria,0.063266
4,Valencia,0.051265
...,...,...
85,Siena,0.000000
86,Lukoil Academic,0.000000
87,Maccabi Tel Aviv,0.000000
88,Bisons Loimaa,0.000000


## xgboost

In [61]:
alg = XGBClassifier(colsample_bytree= 0.5913621206231853, gamma= 4.08026456094463, learning_rate= 0.37903160103981465,
                    max_depth= 47, n_estimators= 94, reg_lambda= 0.11419103539605524, subsample= 0.7917278647223511,
                    objective='binary:logistic').fit(x_train_ec, y_train_ec)

y_pre_proba_alg=alg.predict_proba(x_test_ec)
print("xgboost")
print()
print("Test BS:",round(brier_score_loss(y_test_ec,y_pre_proba_alg[:,1]),3))
print()
y_pre_alg=np.where(y_pre_proba_alg[:,1] > 0.5, 1, 0)
print("Test Accuracy:",round(accuracy_score(y_pre_alg ,y_test_ec),3))
print()
print("Test F1-score:",round(f1_score(y_pre_alg,y_test_ec),3))


xgboost

Test BS: 0.19

Test Accuracy: 0.625

Test F1-score 0.625


In [62]:
coef_dict = {}
for coef, feat in zip(alg.feature_importances_[:].tolist(),ec_x.columns.values):
    coef_dict[feat] = coef
coef3=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef3=coef3.iloc[(-coef3[0].abs()).argsort()].reset_index()
sorted_coef3

,index,0
0,Besiktas,0.038686
1,Krasny Oktyabr,0.032355
2,Lokomotiv Kuban,0.029026
3,Gran Canaria,0.026667
4,Unics Kazan,0.026651
...,...,...
85,Hapoel Jerusalem,0.000000
86,Khimik,0.000000
87,Real Betis,0.000000
88,Rytas Vilnius,0.000000


In [64]:
ens_proba=(y_pre_proba_lgr[:,1]+y_pre_proba_rfc[:,1]+y_pre_proba_alg[:,1])/3


print("------Ensemble 3 classifier-------")
print()
print("Test BS:",round(brier_score_loss(y_test_ec,ens_proba),3))
print()
y_pre_ens=np.where(pd.Series(ens_proba)>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_ens,y_test_ec),3))
print()
print("Test F1-score:",round(f1_score(y_pre_ens,y_test_ec),3))


------Ensemble 3 classifier-------

Test BS: 0.189

Test Accuracy: 0.688

Test F1-score 0.762
